In [1]:
!pip install ratsnlp

In [2]:
from google.colab import drive
drive.mount('/NLPdrive', force_remount=True)

Mounted at /NLPdrive


In [3]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(
    pretrained_model_name='beomi/kcbert-base', # 이전에 사용한 pre-train모델
    downstream_model_dir='/NLPdrive/My Drive/nlpbook/checkpoint-doccls', # 이전에 파인튜닝한 모델의 체크포인트 저장위치
    max_seq_length=128
)

downstream_model_checkpoint_fpath: /NLPdrive/My Drive/nlpbook/checkpoint-doccls/epoch=1-val_loss=0.27.ckpt


In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False
)

In [8]:
# 이전에 파인튜닝한 모델의 체크포인트 읽어오기
import torch

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

In [9]:
# 이전 파인튜닝 때 사용한 pretrained_model_name에 해당하는 모델의 설정값 읽어오기
from transformers import BertConfig

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel()
)

In [10]:
# 설정값대로 BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

In [11]:
# 초기화한 BERT 모델에 체크포인트 주입
model.load_state_dict({k.replace('model.',''):v for k,v in fine_tuned_model_ckpt['state_dict'].items()})

# 모델을 평가 모드로 전환 (드롭아웃 등 학습 때만 사용하는 기법들을 무효화)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [12]:
# 문장을 토큰화 한 뒤 input_ids, attention_mask, token_type_ids 만들기
def inference_fn(sentence):
  inputs = tokenizer(
      [sentence],
      max_length=args.max_seq_length,
      padding='max_length',
      truncation=True
  )

  # 모델 출력값(output.logits)은 소프트맥스 함수 적용 이전의 로짓 형태
  # 따라서 여기에 소프트맥스 함수를 써서 모델 출력을 [부정일 확률, 긍정일 확률]로 바꿈
  with torch.no_grad():
    # input을 파이토치 텐서로 바꿔 모델 계산하기
    outputs = model(**{k: torch.tensor(v) for k,v in inputs.items()})
    prob = outputs.logits.softmax(dim=1) # 로짓에 소프트맥스 취하기

    # 모델 출력을 약간 후처리하여 예측 확률의 최댓값이 부정 위치일 때 해당 문장이 부정, 반대는 긍정이 되도록 pred값 만들기
    positive_prob = round(prob[0][1].item(), 4)
    negative_prob = round(prob[0][0].item(), 4)
    pred = '긍정 (positive)' if torch.argmax(prob) == 1 else '부정 (negativa)'

  return {
      'sentence': sentence,
      'prediction': pred,
      'positive_data': f'긍정 {positive_prob}',
      'negative_data': f'부정 {negative_prob}',
      'positive_width': f'{positive_prob * 100}%',
      'negative_width': f'{negative_prob * 100}%'
  }

In [15]:
# flask 웹 서비스
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.classification.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://2ad6-107-167-183-133.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:12] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:13] "GET /static/EuclidSquare-Regular-WebS.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:13] "GET /static/EuclidSquare-Medium-WebS.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:13] "GET /static/IBMPlexMono-SemiBold.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:13] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:13] "GET /static/IBMPlexMono-SemiBoldItalic.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:13] "GET /static/IBMPlexMono-TextItalic.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:14] "GET /static/IBMPlexMono-Text.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:14] "GET /static/EuclidSquare-MediumItalic-WebS.woff HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Aug/2023 23:58:14] "GET /static/EuclidSquare